In [377]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
import json
import random

In [439]:
10_000 * 1.5

15000.0

In [465]:
3500 / 250

14.0

In [472]:
IMDB_BASE_URL = "https://www.imdb.com"
IMDB_TITLES_URL = "https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&view=simple&sort=user_rating,desc&start={num}"
IMDB_OSCAR_NOMINEES = "https://www.imdb.com/search/title/?title_type=feature&groups=oscar_nominee&view=simple&sort=user_rating,desc&count=250&start={num}"
IMDB_US_MOVIES = "https://www.imdb.com/search/title/?title_type=feature&user_rating=6.0,&num_votes=5000,&countries=us&view=simple&sort=user_rating,desc&count=250&start={num}"
IMDB_GB_MOVIES = "https://www.imdb.com/search/title/?title_type=feature&user_rating=6.0,&num_votes=5000,&countries=gb&view=simple&sort=user_rating,desc&count=250&start={num}"
IMDB_FR_MOVIES = "https://www.imdb.com/search/title/?title_type=feature&user_rating=6.0,&num_votes=5000,&countries=fr&view=simple&sort=user_rating,desc&count=250&start={num}"
IMDB_MOVIE_PAGE_URL = "https://www.imdb.com/title/{movie_id}/"
IMDB_MOVIE_SUMMARY_URL = "https://www.imdb.com/title/{movie_id}/plotsummary"

In [175]:
def get_movies(page):
    soup = BeautifulSoup(page.content, "html.parser")
    movie_divs = soup.find_all("div", class_="col-title")
    movies = dict()
    for div in movie_divs:
        link = div.find('a')
        movies[link.string] = link['href']
    return movies

In [69]:
for page in range(200):
    movies = get_movies(requests.get(IMDB_TITLES_URL.format(num=page * 50 + 1)))
    movies
    with open(f"data/links/{page}.json", "w") as fp:
        json.dump(movies, fp)
    sleep(1)

In [190]:
with open('data/links/0.json', 'r') as fp:
    links = json.load(fp)

In [ ]:
2000 / 250

In [468]:
# getting all oscar nominees
for page in range(14):
    movies = get_movies(requests.get(IMDB_OSCAR_NOMINEES.format(num=page * 250 + 1)))
    with open(f"data/links/oscar_nominees/{page}.json", "w") as fp:
        json.dump(movies, fp)
    sleep(1)

In [471]:
# getting all US movies
for page in range(27):
    movies = get_movies(requests.get(IMDB_US_MOVIES.format(num=page * 250 + 1)))
    with open(f"data/links/us_movies/{page}.json", "w") as fp:
        json.dump(movies, fp)
    sleep(1)

KeyboardInterrupt: 

In [473]:
# getting all GB movies
for page in range(10):
    movies = get_movies(requests.get(IMDB_GB_MOVIES.format(num=page * 250 + 1)))
    with open(f"data/links/gb_movies/{page}.json", "w") as fp:
        json.dump(movies, fp)
    sleep(1)

In [474]:
# getting all FR movies
for page in range(10):
    movies = get_movies(requests.get(IMDB_FR_MOVIES.format(num=page * 250 + 1)))
    with open(f"data/links/fr_movies/{page}.json", "w") as fp:
        json.dump(movies, fp)
    sleep(1)

In [475]:
def get_movie_summaries(movie_id):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
    page = requests.get(
        url=IMDB_MOVIE_SUMMARY_URL.format(movie_id=movie_id),
        headers=headers)
    soup = BeautifulSoup(page.content, "html.parser")
    summary_section = soup.find("div", attrs={'data-testid': 'sub-section-summaries'})
    summaries = list()
    for summary in summary_section.find_all('li'):
        summaries.append(summary.get_text())
    return summaries

In [476]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
page = requests.get(
    url=IMDB_MOVIE_PAGE_URL.format(movie_id='tt0402901'),
    headers=headers)
soup = BeautifulSoup(page.content, "html.parser")

In [477]:
# Movie details to pull: 
# - Movie Name
# - Director
# - Top actors
# - Genre
# - Rating
# - Plot
# - Release Date
# - Synopsis
# - Run time
# - Poster
# - Certificate

def get_movie_details(movie_id):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
    page = requests.get(
        url=IMDB_MOVIE_PAGE_URL.format(movie_id=movie_id),
        headers=headers)
    soup = BeautifulSoup(page.content, "html.parser")
    # title
    title = soup.find('h1', attrs={'data-testid': 'hero__pageTitle'}).get_text()
    # genre
    genres = ', '.join([a.get_text() for a in soup.find('div', attrs={'data-testid': 'genres'}).find_all('a')])
    # plot 
    plot = soup.find('p', attrs={'data-testid': 'plot'}).span.get_text()
    # director, top actors
    ul = soup.find("ul", class_="ipc-metadata-list ipc-metadata-list--dividers-all title-pc-list ipc-metadata-list--baseAlt")
    director, writers, stars = ul.find_all('li', attrs={'data-testid': 'title-pc-principal-credit'}, recursive=False)
    starring = ', '.join([li.text for li in stars.ul.find_all('li', recursive=False)])
    director = director.ul.li.text
    # rating
    rating = soup.find('div', attrs={'data-testid': 'hero-rating-bar__aggregate-rating__score'}).span.text
    # release date
    release_date = soup.find('li', attrs={'data-testid': 'title-details-releasedate'}).div.ul.li.text
    # run time
    runtime = soup.find('li', attrs={'data-testid': 'title-techspec_runtime'}).div.text
    # certificate
    certificate = soup.find('div', class_='sc-52d569c6-0 kNzJA-D').ul.find_all('li')[1].text
    # img url 
    img_url = soup.find('div', attrs={'data-testid': 'hero-media__poster'}).img['srcset'].split(', ')[-1].split(' ')[0]
    return {
        'title': title,
        'genres': genres,
        'director': director,
        'starring': starring,
        'rating': rating,
        'release_date': release_date,
        'runtime': runtime,
        'certificate': certificate,
        'img_url': img_url,
        'plot': plot
    }
    

In [478]:
def id_from_link(link):
    return link.strip('/').split('/')[1]

def folder_name_from_link(link, name):
    lower_name = "_".join([word.lower() for word in name.split()])
    return f"{id_from_link(link)}_{lower_name}"

In [482]:
for links_path in glob.glob('data/links/*/*.json'):
    # load a batch of links
    with open(links_path, 'r') as fp:
        links = json.load(fp)

    # process links
    for name, link in links.items():
        print(f"Getting summary for {name}.")
        movie_id = id_from_link(link)
        movie_folder_name = folder_name_from_link(link, name)
        movie_path = f'data/movies/{movie_folder_name}'
        movie_summaries_path = f'data/movies/{movie_folder_name}/plot_summaries'
        
        # create folder for movie if necessary
        if not os.path.exists(movie_path):
            os.makedirs(movie_path)
            os.makedirs(movie_summaries_path)
        # already exists, skip this one
        else:
            continue
        
        # get the data
        try:
            movie_details = get_movie_details(movie_id)
            sleep(random.random())
            movie_summaries = get_movie_summaries(movie_id)
        except:
            continue
        
        # write the movie details to disk
        with open(f"{movie_path}/summary.json", "w") as fp:
            json.dump(movie_details, fp)
        
        # write the movie poster to disk
        img = Image.open(requests.get(movie_details['img_url'], stream = True).raw)
        img.save(f"{movie_path}/poster.png")

        # write the movie plot summaries to disk 
        for i, summary in enumerate(movie_summaries):
            with open(f"{movie_summaries_path}/{i}.txt", 'w') as fp:
                fp.write(summary)
        
        # take a break
        sleep_time = random.random() * 3
        print(f"Sleeping for {sleep_time:.2f}s")
        sleep(sleep_time)

Getting summary for City of God.
Getting summary for Léon: The Professional.
Getting summary for The Pianist.
Getting summary for The Intouchables.
Getting summary for Cinema Paradiso.
Getting summary for Le Trou.
Getting summary for Michael the Brave.
Getting summary for Capernaum.
Getting summary for Amadeus.
Getting summary for The Lives of Others.
Getting summary for Das Boot.
Getting summary for Filantropica.
Getting summary for Guardians of the Galaxy Vol. 3.
Getting summary for Incendies.
Getting summary for Amélie.
Getting summary for A Separation.
Getting summary for Children of Paradise.
Getting summary for The Father.
Getting summary for Casino.
Getting summary for Ran.
Getting summary for Z.
Getting summary for The Passion of Joan of Arc.
Getting summary for The Wages of Fear.
Getting summary for Rocco and His Brothers.
Getting summary for A Man Escaped.
Getting summary for Napoleon.
Getting summary for How to Train Your Dragon.
Getting summary for Wild Tales.
Getting summa

Sleeping for 1.55s
Getting summary for The Secret of the Grain.
Sleeping for 2.39s
Getting summary for Divines.
Sleeping for 0.15s
Getting summary for The Milky Way.
Sleeping for 0.96s
Getting summary for Antonia's Line.
Sleeping for 2.12s
Getting summary for Autumn Tale.
Sleeping for 1.92s
Getting summary for The Toy.
Sleeping for 2.57s
Getting summary for The Trial of Joan of Arc.
Sleeping for 1.98s
Getting summary for What Will People Say.
Sleeping for 2.41s
Getting summary for Bed & Board.
Sleeping for 0.31s
Getting summary for A Town Called Panic.
Sleeping for 1.16s
Getting summary for French Fried Vacation 2.
Sleeping for 0.40s
Getting summary for Seraphine.
Sleeping for 2.18s
Getting summary for Ten.
Sleeping for 0.54s
Getting summary for Clash.
Sleeping for 2.44s
Getting summary for Family Resemblances.
Sleeping for 0.88s
Getting summary for The Dreamlife of Angels.
Sleeping for 1.14s
Getting summary for East/West.
Sleeping for 2.77s
Getting summary for Triangle of Sadness.
Get

Getting summary for Dear Diary.
Sleeping for 1.46s
Getting summary for Dheepan.
Sleeping for 1.31s
Getting summary for Confidentially Yours.
Sleeping for 2.25s
Getting summary for 99 francs.
Sleeping for 0.37s
Getting summary for A Tale of Winter.
Sleeping for 2.57s
Getting summary for Romanzo Criminale.
Sleeping for 0.73s
Getting summary for The Taste of Others.
Sleeping for 1.51s
Getting summary for Delusions of Grandeur.
Sleeping for 0.55s
Getting summary for Les trois frères.
Sleeping for 2.44s
Getting summary for John Rabe.
Sleeping for 2.75s
Getting summary for Buffet Froid.
Sleeping for 1.30s
Getting summary for Asterix and Cleopatra.
Sleeping for 0.11s
Getting summary for Cyclo.
Sleeping for 1.98s
Getting summary for I Hired a Contract Killer.
Sleeping for 0.90s
Getting summary for Like Crazy.
Sleeping for 0.29s
Getting summary for Jour de Fête.
Sleeping for 2.08s
Getting summary for Foxtrot.
Sleeping for 2.80s
Getting summary for Two English Girls.
Sleeping for 0.57s
Getting s

Getting summary for Millennium Mambo.
Sleeping for 0.16s
Getting summary for Indochine.
Sleeping for 1.72s
Getting summary for Priceless.
Getting summary for Donkey Skin.
Sleeping for 0.95s
Getting summary for High Heels.
Sleeping for 0.07s
Getting summary for Involuntary.
Sleeping for 1.36s
Getting summary for Heaven.
Getting summary for The House of Mirth.
Sleeping for 2.08s
Getting summary for La Ciénaga.
Sleeping for 0.96s
Getting summary for A War.
Sleeping for 1.16s
Getting summary for Jiang hu er nü.
Sleeping for 0.68s
Getting summary for Angel-A.
Getting summary for Paradise: Love.
Sleeping for 2.63s
Getting summary for Two Brothers.
Sleeping for 1.24s
Getting summary for Girlhood.
Sleeping for 2.38s
Getting summary for The Lady.
Sleeping for 2.36s
Getting summary for Now Is Good.
Getting summary for Breathe.
Sleeping for 0.38s
Getting summary for Julia.
Sleeping for 1.45s
Getting summary for Days of Glory.
Sleeping for 1.61s
Getting summary for Goodbye Berlin.
Sleeping for 1.3

Getting summary for Point Blank.
Sleeping for 0.76s
Getting summary for Neruda.
Sleeping for 2.26s
Getting summary for Paris.
Sleeping for 1.04s
Getting summary for Populaire.
Sleeping for 0.41s
Getting summary for Back to Burgundy.
Sleeping for 2.28s
Getting summary for Romantics Anonymous.
Sleeping for 1.79s
Getting summary for Wilbur Wants to Kill Himself.
Sleeping for 2.31s
Getting summary for Under the Tree.
Sleeping for 1.51s
Getting summary for 101 Reykjavík.
Sleeping for 2.83s
Getting summary for Mia madre.
Sleeping for 0.92s
Getting summary for Dikkenek.
Sleeping for 2.27s
Getting summary for Fantomas Unleashed.
Sleeping for 0.13s
Getting summary for Kandahar.
Sleeping for 1.65s
Getting summary for Lights in the Dusk.
Sleeping for 1.64s
Getting summary for The Measure of a Man.
Sleeping for 1.54s
Getting summary for Asterix Versus Caesar.
Sleeping for 1.58s
Getting summary for Carancho.
Sleeping for 2.54s
Getting summary for Vincere.
Sleeping for 2.78s
Getting summary for The 

...

Sleeping for 0.65s
Getting summary for Patch Adams.
Getting summary for Carmen Jones.
Getting summary for All the Money in the World.
Getting summary for Brother Bear.
Getting summary for Working Girl.
Getting summary for Judy.
Getting summary for Life.
Getting summary for The Crucible.
Getting summary for The American President.
Getting summary for Innerspace.
Getting summary for Elizabeth: The Golden Age.
Getting summary for The Alamo.
Getting summary for The Ghost and the Darkness.
Getting summary for Bugsy Malone.
Getting summary for Victoria & Abdul.
Getting summary for South Pacific.
Getting summary for Fantastic Voyage.
Getting summary for Hitchcock.
Getting summary for Anonymous.
Getting summary for Bugsy.
Getting summary for Florence Foster Jenkins.
Getting summary for Young Sherlock Holmes.
Getting summary for Fellini Satyricon.
Getting summary for The Prince of Tides.
Getting summary for The Hot Rock.
Getting summary for The Muppets Take Manhattan.
Getting summary for Omar.


Sleeping for 0.25s
Getting summary for So Proudly We Hail!.
Sleeping for 1.69s
Getting summary for No Way Out.
Getting summary for 'Round Midnight.
Getting summary for Tristana.
Getting summary for The Gay Divorcee.
Getting summary for The Cruel Sea.
Getting summary for Paradise Now.
Getting summary for David Copperfield.
Sleeping for 1.78s
Getting summary for Champion.
Sleeping for 0.41s
Getting summary for Twin Sisters.
Sleeping for 2.08s
Getting summary for The Firemen's Ball.
Sleeping for 0.53s
Getting summary for Cyrano de Bergerac.
Sleeping for 1.38s
Getting summary for The Informer.
Getting summary for Antonia's Line.
Getting summary for Strawberry & Chocolate.
Sleeping for 2.92s
Getting summary for Sitting Pretty.
Sleeping for 0.32s
Getting summary for Tabu: A Story of the South Seas.
Getting summary for The Return of Martin Guerre.
Sleeping for 1.66s
Getting summary for Executive Suite.
Sleeping for 2.45s
Getting summary for The Visit.
Sleeping for 0.77s
Getting summary for Ge

Sleeping for 0.32s
Getting summary for The Secret of Santa Vittoria.
Sleeping for 0.14s
Getting summary for Hilary and Jackie.
Getting summary for Passion Fish.
Getting summary for Camille.
Getting summary for The Westerner.
Getting summary for Holiday Inn.
Getting summary for The Naked Prey.
Getting summary for My Favorite Wife.
Getting summary for Marie Antoinette.
Sleeping for 1.39s
Getting summary for The Front.
Getting summary for Objective, Burma!.
Getting summary for Five Graves to Cairo.
Getting summary for The Tin Star.
Getting summary for A Thousand Clowns.
Sleeping for 0.62s
Getting summary for In Darkness.
Sleeping for 0.06s
Getting summary for Never on Sunday.
Sleeping for 1.09s
Getting summary for Lili.
Sleeping for 0.58s
Getting summary for Love Affair.
Getting summary for A Boy Named Charlie Brown.
Getting summary for A Foreign Affair.
Getting summary for The Garden of the Finzi-Continis.
Sleeping for 0.37s
Getting summary for Hold Back the Dawn.
Sleeping for 0.83s
Gett

Getting summary for Smart Money.
Sleeping for 1.49s
Getting summary for Hot Millions.
Sleeping for 2.26s
Getting summary for The Captain's Paradise.
Sleeping for 1.75s
Getting summary for Gaby: A True Story.
Sleeping for 2.67s
Getting summary for The President's Lady.
Sleeping for 2.00s
Getting summary for The Princess and the Pirate.
Sleeping for 0.70s
Getting summary for Martin Luther.
Sleeping for 1.52s
Getting summary for Good News.
Sleeping for 1.20s
Getting summary for Strike Up the Band.
Sleeping for 1.79s
Getting summary for Solomon & Gaenor.
Sleeping for 0.86s
Getting summary for No Time for Love.
Sleeping for 2.31s
Getting summary for Primrose Path.
Sleeping for 1.39s
Getting summary for Trial.
Sleeping for 0.26s
Getting summary for Perri.
Sleeping for 2.69s
Getting summary for Orchestra Wives.
Sleeping for 2.71s
Getting summary for Of Human Hearts.
Sleeping for 1.24s
Getting summary for Souls at Sea.
Sleeping for 0.99s
Getting summary for Test Pilot.
Sleeping for 0.19s
Getti

Sleeping for 2.81s
Getting summary for When Tomorrow Comes.
Sleeping for 1.59s
Getting summary for Hallelujah.
Sleeping for 2.99s
Getting summary for Our Very Own.
Sleeping for 1.79s
Getting summary for Back Street.
Sleeping for 0.57s
Getting summary for Hi Diddle Diddle.
Sleeping for 1.15s
Getting summary for Qivitoq.
Sleeping for 2.14s
Getting summary for Lies My Father Told Me.
Sleeping for 2.05s


In [ ]:


with open('data/links/0.json', 'r') as fp:
    links = json.load(fp)

In [134]:
summaries = get_movie_summaries(movie_summary_page)

In [139]:
summaries[3]

"After being anointed as the next king of the jungle, young lion cub Simba is every bit as eager to take his proud father Mufasa's place as the king. However, this makes Simba's villainous uncle Scar jealous of him. But when Mufasa is suddenly murdered by Scar, Simba feels responsible for his death and runs away from home. Meeting up with two outcasts named Timon and Pumbaa, Simba embraces their ways of 'Hakuna Matata'. Years later, approached by his childhood friend Nala and the wise baboon Rafiki, Simba must return home to stop Scar's evil reign and fulfill his destiny as the new king.—Blazer346"

In [108]:
soup = BeautifulSoup(movie_summary.content, "html.parser")

In [459]:
summaries = []
for summary_path in glob.glob('data/movies/*/summary.json'):
    imdb_id = summary_path.split('/')[2].split('_')[0]
    with open(summary_path, 'r') as fp:
        summary = json.load(fp)
        summary['id'] = imdb_id
        summaries.append(summary)

In [447]:
import pandas as pd

In [460]:
df = pd.DataFrame(summaries)

In [463]:
df.certificate.value_counts()[:10]

certificate
R            2533
PG-13        1629
PG            800
Not Rated     422
G             150
TV-MA          97
Passed         56
Approved       49
TV-14          31
NC-17          19
Name: count, dtype: int64